In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [11]:
###change path below####
path = 'chi_example.csv' 

data = pd.read_csv(path)
data.set_index(['ab_test_name'],inplace=True)
data

,devices,opt_ins,not_opt_ins
ab_test_name,,,
Control,249135,224,248911
upsell,621864,1012,620852
upsell_discout_churn,621100,987,620113


In [12]:
data_stats = data[['opt_ins', 'not_opt_ins']].transpose()
data_stats

ab_test_name,Control,upsell,upsell_discout_churn
opt_ins,224,1012,987
not_opt_ins,248911,620852,620113


In [35]:

############################################################

####enter parameters below and do chi_square analysis#######

############################################################

df = data_stats
control_name = 'Control' # change to your experiment group name
test_name = 'upsell'# change to your experiment group name
metric = 'opt-ins rate' #change to your metric name


############################################################
######chi_2 analysis(no need to change below)###############
############################################################

observed_1 = data_stats [[control_name ,test_name]] 
chi_square, p_value, degree_of_freedom, ob_df = stats.chi2_contingency(observed= observed_1)

if p_value < 0.05:
    sig = 'significant'
else:
    sig = 'not significant'

start = "\033[1m"
end = "\033[0;0m"


print 'chi square results below:'
print ''
print '############################################################'
print ''
print '     Result for this experiment is %s' % (start + sig.upper() + end)
print ''
print '1) the p_value for %s is: %.5f' % (metric, p_value)
print '2) chi_square for %s is %.2f' % (metric, chi_square)
print '3) degree of freedom for %s is %d'  % (metric, degree_of_freedom)
print ''
print '############################################################'


chi square results below:

############################################################

     Result for this experiment is SIGNIFICANT

1) the p_value for opt-ins rate is: 0.00000
2) chi_square for opt-ins rate is 66.06
3) degree of freedom for opt-ins rate is 1

############################################################


23.82

# additional notes: 


**  1) for the observed dataframe, it doesn't matter if you transpose or not. **

In [ ]:
# practice(transposing contingence table or not doesn't matter to the final chi2 test result, it is the same)

In [4]:
np.random.seed(10)

# Sample data randomly at fixed probabilities
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"],
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Sample data randomly at fixed probabilities
voter_party = np.random.choice(a= ["democrat","independent","republican"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

voters = pd.DataFrame({"race":voter_race, 
                       "party":voter_party})

voter_tab = pd.crosstab(voters.race, voters.party, margins = True)

voter_tab.columns = ["democrat","independent","republican","row_totals"]

voter_tab.index = ["asian","black","hispanic","other","white","col_totals"]

observed = voter_tab.ix[0:5,0:3]   # Get table without totals for later use
voter_tab

/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,democrat,independent,republican,row_totals
asian,21,7,32,60
black,65,25,64,154
hispanic,107,50,94,251
other,15,8,15,38
white,189,96,212,497
col_totals,397,186,417,1000


In [5]:
expected =  np.outer(voter_tab["row_totals"][0:5],
                     voter_tab.ix["col_totals"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["democrat","independent","republican"]
expected.index = ["asian","black","hispanic","other","white"]

expected

,democrat,independent,republican
asian,23,11,25
black,61,28,64
hispanic,99,46,104
other,15,7,15
white,197,92,207


In [6]:
observed

,democrat,independent,republican
asian,21,7,32
black,65,25,64
hispanic,107,50,94
other,15,8,15
white,189,96,212


In [7]:
stats.chi2_contingency(observed= observed)

(7.1693212801620589,
 0.51847939294884204,
 8,
 array([[  23.82 ,   11.16 ,   25.02 ],
        [  61.138,   28.644,   64.218],
        [  99.647,   46.686,  104.667],
        [  15.086,    7.068,   15.846],
        [ 197.309,   92.442,  207.249]]))

In [9]:
observed2 = observed.transpose()
observed2

,asian,black,hispanic,other,white
democrat,21,65,107,15,189
independent,7,25,50,8,96
republican,32,64,94,15,212


In [10]:
stats.chi2_contingency(observed= observed2)

(7.1693212801620598,
 0.51847939294884204,
 8,
 array([[  23.82 ,   61.138,   99.647,   15.086,  197.309],
        [  11.16 ,   28.644,   46.686,    7.068,   92.442],
        [  25.02 ,   64.218,  104.667,   15.846,  207.249]]))

 ** 2). parametric (Z-test) and non-parametric (chi-squared) methods for proportion test **

Very good reading 
 - **Comparison of two proportions: parametric (Z-test) and non-parametric (chi-squared) methods **
   - if we assume the probability of common success is approximate 0.5, and the number of games is very high (under these assumption, a binomial distribution is approximate a gaussian distribution). Then we can use z-test to analyze two proportions. Otherwise, just use chi-squared test.
   - A chi-square test for equality of two proportions is exactly the same thing as a zz-test, if we have the two assumptions above.
   
   - https://www.r-bloggers.com/comparison-of-two-proportions-parametric-z-test-and-non-parametric-chi-squared-methods/
    

In [4]:
from scipy import stats
z_scores = -2.969695
p_values = stats.norm.sf(abs(z_scores)) #one-sided

p_values


0.0014904777600325016

- this two-sided p_values equal to use the p-value using stats.chi2_contingency(observed= observed_1) in the R-blog example

In [6]:
p_values = stats.norm.sf(abs(z_scores))*2 #twosided
p_values

0.0029809555200650032